In [ ]:
# | default_exp baselines


In [ ]:
# |export
from abc import ABC, abstractmethod
from typing import Iterable

import gpflow
import numpy as np
import tensorflow as tf
from gpflow.mean_functions import Constant
from gpflow.utilities import positive, print_summary
from gpflow.utilities.ops import broadcasting_elementwise
from nbdev.showdoc import *
from optuna import create_study
from optuna.integration import XGBoostPruningCallback
from optuna.samplers import TPESampler
from rdkit.Chem import AllChem, Descriptors, MolFromSmiles, MolToSmiles
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from wandb.xgboost import WandbCallback
from xgboost import XGBClassifier, XGBRegressor

/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Baselines

> Code for baseline models.


In [ ]:
# |export


class BaseLineModel(ABC):
    @abstractmethod
    def tune(self, X_train, y_train):
        raise NotImplementedError()

    @abstractmethod
    def fit(self, X_train, y_train):
        raise NotImplementedError()

    @abstractmethod
    def predict(self, X):
        raise


In [ ]:
# |export


class XGBClassificationBaseline(BaseLineModel):
    def __init__(self, seed, num_trials=100) -> None:
        self.seed = seed
        self.num_trials = num_trials
        self.model = XGBClassifier()

        self.label_encoder = LabelEncoder()

    def tune(self, X_train, y_train):
        y_train = self.label_encoder.fit_transform(y_train)

        def objective(
            trial,
            X,
            y,
            random_state=22,
            n_splits=3,
            n_jobs=1,
            early_stopping_rounds=100,
        ):
            # XGBoost parameters
            params = {
                "verbosity": 0,  # 0 (silent) - 3 (debug)
                "n_estimators": trial.suggest_int("n_estimators", 4, 10_000),
                "max_depth": trial.suggest_int("max_depth", 4, 100),
                "learning_rate": trial.suggest_loguniform("learning_rate", 0.001, 0.05),
                "colsample_bytree": trial.suggest_loguniform(
                    "colsample_bytree", 0.2, 1
                ),
                "subsample": trial.suggest_loguniform("subsample", 0.00001, 1),
                "alpha": trial.suggest_loguniform("alpha", 1e-8, 10.0),
                "lambda": trial.suggest_loguniform("lambda", 1e-8, 10.0),
                "seed": random_state,
                "n_jobs": n_jobs,
            }

            model = XGBClassifier(**params)
            pruning_callback = XGBoostPruningCallback(trial, "validation_0-mlogloss")
            kf = KFold(n_splits=n_splits)
            X_values = X.values
            y_values = y

            scores = []
            for train_index, test_index in kf.split(X_values):
                X_A, X_B = X_values[train_index, :], X_values[test_index, :]
                y_A, y_B = y_values[train_index], y_values[test_index]

                model.fit(
                    X_A,
                    y_A,
                    eval_set=[(X_B, y_B)],
                    eval_metric="mlogloss",
                    verbose=0,
                    callbacks=[pruning_callback],
                    early_stopping_rounds=early_stopping_rounds,
                )
                y_pred = model.predict(X_B)
                scores.append(f1_score(y_pred, y_B, average="macro"))
            return np.mean(scores)

        sampler = TPESampler(seed=self.seed)
        study = create_study(direction="maximize", sampler=sampler)
        study.optimize(
            lambda trial: objective(
                trial,
                X_train,
                y_train,
                random_state=self.seed,
                n_splits=5,
                n_jobs=-1,
                early_stopping_rounds=100,
            ),
            n_trials=self.num_trials,
            n_jobs=1,
        )

        self.model = XGBClassifier(**study.best_params, callbacks=[WandbCallback()])

    def fit(self, X_train, y_train):
        y_train = self.label_encoder.fit_transform(y_train)
        self.model.fit(X_train, y_train)

    def predict(self, X):
        return self.label_encoder.inverse_transform(self.model.predict(X))


In [ ]:
# |export
class XGBRegressionBaseline(BaseLineModel):
    def __init__(self, seed, num_trials=100) -> None:
        self.seed = seed
        self.num_trials = num_trials
        self.model = XGBRegressor()

    def tune(self, X_train, y_train):
        def objective(
            trial,
            X,
            y,
            random_state=22,
            n_splits=3,
            n_jobs=1,
            early_stopping_rounds=50,
        ):
            # XGBoost parameters
            params = {
                "verbosity": 0,  # 0 (silent) - 3 (debug)
                "objective": "reg:squarederror",
                "n_estimators": 10000,
                "max_depth": trial.suggest_int("max_depth", 4, 12),
                "learning_rate": trial.suggest_loguniform("learning_rate", 0.005, 0.05),
                "colsample_bytree": trial.suggest_loguniform(
                    "colsample_bytree", 0.2, 0.6
                ),
                "subsample": trial.suggest_loguniform("subsample", 0.4, 0.8),
                "alpha": trial.suggest_loguniform("alpha", 0.01, 10.0),
                "lambda": trial.suggest_loguniform("lambda", 1e-8, 10.0),
                "gamma": trial.suggest_loguniform("lambda", 1e-8, 10.0),
                "min_child_weight": trial.suggest_loguniform(
                    "min_child_weight", 10, 1000
                ),
                "seed": random_state,
                "n_jobs": n_jobs,
            }

            model = XGBRegressor(**params)
            pruning_callback = XGBoostPruningCallback(trial, "validation_0-rmse")
            kf = KFold(n_splits=n_splits)
            X_values = X.values
            y_values = y.values
            scores = []
            for train_index, test_index in kf.split(X_values):
                X_A, X_B = X_values[train_index, :], X_values[test_index, :]
                y_A, y_B = y_values[train_index], y_values[test_index]
                model.fit(
                    X_A,
                    y_A,
                    eval_set=[(X_B, y_B)],
                    eval_metric="rmse",
                    verbose=0,
                    callbacks=[pruning_callback],
                    early_stopping_rounds=early_stopping_rounds,
                )
                y_pred = model.predict(X_B)
                scores.append(mean_squared_error(y_pred, y_B))
            return np.mean(scores)

        sampler = TPESampler(seed=self.seed)
        study = create_study(direction="minimize", sampler=sampler)
        study.optimize(
            lambda trial: objective(
                trial,
                X_train,
                y_train,
                random_state=self.seed,
                n_splits=5,
                n_jobs=8,
                early_stopping_rounds=100,
            ),
            n_trials=self.num_trials,
            n_jobs=1,
        )

        self.model = XGBRegressor(**study.best_params)

    def fit(self, X_train, y_train):
        self.model.fit(X_train, y_train)

    def predict(self, X):
        return self.model.predict(X)


## Photoswitch

> Code specific for the photoswitch test case


For the photoswitch datataset, we'll use a GPR on the "fragprint" representation using a Tanimoto kernel (as in [the original implementation](https://github.com/Ryan-Rhys/The-Photoswitch-Dataset/blob/master/property_prediction/predict_with_GPR.py))


In [ ]:
# |export



class Tanimoto(gpflow.kernels.Kernel):
    """Tanimoto kernel.

    Taken from https://github.com/Ryan-Rhys/The-Photoswitch-Dataset/blob/master/property_prediction/kernels.py.
    """

    def __init__(self, **kwargs):
        """
        :param kwargs: accepts `name` and `active_dims`, which is a list or
            slice of indices which controls which columns of X are used (by
            default, all columns are used).
        """
        for kwarg in kwargs:
            if kwarg not in {"name", "active_dims"}:
                raise TypeError("Unknown keyword argument:", kwarg)
        super().__init__(**kwargs)
        self.variance = gpflow.Parameter(1.0, transform=positive())

    def K(self, X, X2=None):
        """
        Compute the Tanimoto kernel matrix σ² * ((<x, y>) / (||x||^2 + ||y||^2 - <x, y>))
        :param X: N x D array
        :param X2: M x D array. If None, compute the N x N kernel matrix for X.
        :return: The kernel matrix of dimension N x M
        """
        if X2 is None:
            X2 = X

        Xs = tf.reduce_sum(tf.square(X), axis=-1)  # Squared L2-norm of X
        X2s = tf.reduce_sum(tf.square(X2), axis=-1)  # Squared L2-norm of X2
        cross_product = tf.tensordot(
            X, X2, [[-1], [-1]]
        )  # outer product of the matrices X and X2

        # Analogue of denominator in Tanimoto formula

        denominator = -cross_product + broadcasting_elementwise(tf.add, Xs, X2s)

        return self.variance * cross_product / denominator

    def K_diag(self, X):
        """
        Compute the diagonal of the N x N kernel matrix of X
        :param X: N x D array
        :return: N x 1 array
        """
        return tf.fill(tf.shape(X)[:-1], tf.squeeze(self.variance))


In [ ]:
# |export


def compute_fragprints(
    smiles_list: Iterable[str] # list of SMILEs
) -> np.ndarray:
    rdkit_mols = [MolFromSmiles(smiles) for smiles in smiles_list]
    rdkit_smiles = [MolToSmiles(mol, isomericSmiles=False) for mol in rdkit_mols]
    rdkit_mols = [MolFromSmiles(smiles) for smiles in rdkit_smiles]
    X = [
        AllChem.GetMorganFingerprintAsBitVect(mol, 3, nBits=2048)
        for mol in rdkit_mols
    ]
    X = np.asarray(X)

    fragments = {d[0]: d[1] for d in Descriptors.descList[115:]}
    X1 = np.zeros((len(smiles_list), len(fragments)))
    for i in range(len(smiles_list)):
        mol = MolFromSmiles(smiles_list[i])
        try:
            features = [fragments[d](mol) for d in fragments]
        except:
            raise Exception("molecule {}".format(i) + " is not canonicalised")
        X1[i, :] = features

    X = np.concatenate((X, X1), axis=1)
    return X

In [ ]:
# |export

class GPRBaseline(BaseLineModel):
    """GPR w/ Tanimoto kernel baseline."""
    def __init__(self) -> None:
        self.model = None
        self.y_scaler = StandardScaler()

    def tune(
        self,
        X_train: np.ndarray, # N x D features
        y_train: np.ndarray  # N x 1 target
    ):
        pass

    def fit(
        self,
        X_train: np.ndarray, # N x D features
        y_train: np.ndarray  # N x 1 target
    ):
        y_train = y_train.reshape(-1, 1)

        def objective_closure():
            return -m.log_marginal_likelihood()

        y_train = self.y_scaler.fit_transform(y_train)

        m = gpflow.models.GPR(
            data=(X_train, y_train),
            mean_function=Constant(np.mean(y_train)),
            kernel=Tanimoto(),
            noise_variance=1,
        )

        # Optimise the kernel variance and noise level by the marginal likelihood
        opt = gpflow.optimizers.Scipy()
        opt.minimize(
            objective_closure, m.trainable_variables, options=dict(maxiter=10000)
        )
        print_summary(m)
        self.model = m

    def predict(
            self, 
            X_test: np.ndarray # N x D features
        ):  
        y_pred, y_var = self.model.predict_f(X_test)
        y_pred = self.y_scaler.inverse_transform(y_pred)
        return y_pred


In [ ]:
from sklearn.model_selection import train_test_split
from gpt3forchem.data import get_photoswitch_data
from sklearn.metrics import r2_score

Let's create some data using "fragprint" features

In [ ]:
df = get_photoswitch_data()
smiles_list = df['SMILES'].values
y = df['E isomer pi-pi* wavelength in nm'].values
X = compute_fragprints(smiles_list)

Get a random train/test split. In the original work they use a random, unstratified split in 80/20 ratio.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Now, let's run the baseline

In [ ]:
baseline = GPRBaseline()
baseline.fit(X_train, y_train)

/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/gpflow/experimental/utils.py:42: UserWarning: You're calling gpflow.experimental.check_shapes.checker.ShapeChecker.__init__ which is considered *experimental*. Expect: breaking changes, poor documentation, and bugs.
  warn(
2022-08-31 13:11:44.145813: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


╒═════════════════════════╤═══════════╤══════════════════╤═════════╤═════════════╤═════════╤═════════╤══════════╕
│ name                    │ class     │ transform        │ prior   │ trainable   │ shape   │ dtype   │    value │
╞═════════════════════════╪═══════════╪══════════════════╪═════════╪═════════════╪═════════╪═════════╪══════════╡
│ GPR.mean_function.c     │ Parameter │ Identity         │         │ True        │ ()      │ float64 │ -0.0388  │
├─────────────────────────┼───────────┼──────────────────┼─────────┼─────────────┼─────────┼─────────┼──────────┤
│ GPR.kernel.variance     │ Parameter │ Softplus         │         │ True        │ ()      │ float64 │ 41.8636  │
├─────────────────────────┼───────────┼──────────────────┼─────────┼─────────────┼─────────┼─────────┼──────────┤
│ GPR.likelihood.variance │ Parameter │ Softplus + Shift │         │ True        │ ()      │ float64 │  0.02649 │
╘═════════════════════════╧═══════════╧══════════════════╧═════════╧═════════════╧══════

In [ ]:
predictions = baseline.predict(X_test)

In [ ]:
r2_score(y_test, predictions)

0.9207905563562994

This seems quite comparable to the results from the original paper (0.9 stated in the [README](https://github.com/Ryan-Rhys/The-Photoswitch-Dataset)).